In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## Load MNIST Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# these are numpy arrays

print(x_train.shape)
print(y_train.shape)

x_train = x_train.reshape(-1, 28*28)
# change float64 to float32 for lower computation cost
x_train = x_train.astype('float32')
x_train = x_train / 255.0


x_test = x_test.reshape(-1, 28*28).astype('float32')/255.0
# x_trian = tf.convert_to_tensor(x_train)
# we don't need to worry, conversion will happen automatically

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)


## Sequential API (Very convenient, not very flexible) 
=> only 1 input and 1 output

In [ ]:
model = keras.Sequential([
      layers.Dense(512, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(10),
])

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

# print(model.summary()) ==== Can't print here

# verbose 0=>silent,1=>progress bar/interactive,2=>one line per epoch
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

print(model.summary())

Epoch 1/5
1875/1875 - 8s - loss: 0.1886 - accuracy: 0.9437
Epoch 2/5
1875/1875 - 8s - loss: 0.0803 - accuracy: 0.9752
Epoch 3/5
1875/1875 - 8s - loss: 0.0566 - accuracy: 0.9826
Epoch 4/5
1875/1875 - 8s - loss: 0.0417 - accuracy: 0.9867
Epoch 5/5
1875/1875 - 8s - loss: 0.0337 - accuracy: 0.9892
313/313 - 1s - loss: 0.0728 - accuracy: 0.9815
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_18 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model = keras.Sequential([
      layers.Input((28*28)),
      layers.Dense(512, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(10),
])

print(model.summary())


# Large models checking layer sizes
model = keras.Sequential()
model.add(layers.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
print(model.summary())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_27 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_28 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 512)               401920    
Total params: 401,920
Trainable params: 401,920
Non-trainable params: 0
_________________________________________________________________
None
Model: "se

## Functional API (A bit more flexible)

In [ ]:
inputs = layers.Input(shape=(28*28))
x = layers.Dense(512, activation='relu', name='first_layer')(inputs)
x = layers.Dense(256, activation='relu', name='second_layer')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
first_layer (Dense)          (None, 512)               401920    
_________________________________________________________________
second_layer (Dense)         (None, 256)               131328    
_________________________________________________________________
dense_34 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=32)

Epoch 1/5
1875/1875 - 8s - loss: 0.1859 - accuracy: 0.9430
Epoch 2/5
1875/1875 - 8s - loss: 0.0788 - accuracy: 0.9754
Epoch 3/5
1875/1875 - 8s - loss: 0.0538 - accuracy: 0.9829
Epoch 4/5
1875/1875 - 8s - loss: 0.0429 - accuracy: 0.9860
Epoch 5/5
1875/1875 - 8s - loss: 0.0322 - accuracy: 0.9890
313/313 [==============================] - 1s 2ms/step - loss: 0.0700 - accuracy: 0.9815


[0.07001225650310516, 0.9815000295639038]

Functional API with 2 outputs e.g. multi digit mnist

In [ ]:
#======  HyperParameters  ==============
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [ ]:
inputs = keras.Input(shape=(64,64,1))

x = layers.Conv2D(32, 3, padding='same', kernel_regularizer=
                  regularizers.l2(WEIGHT_DECAY))(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)

x = layers.Conv2D(64, 3, activation='relu', kernel_regularizer=
                  regularizers.l2(WEIGHT_DECAY))(x)
x = layers.MaxPooling2D()(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)

#====================================================
# first_num and second_num will be label in outputs in dataset
output1 = layers.Dense(10, activation='softmax', name='first_num')(x)
output2 = layers.Dense(10, activation='softmax', name='second_num')(x)
model = keras.Model(inputs=inputs, outputs=[output1, output2])

#====================================================

model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

model.fit(train_dataset, epochs=5, verbose=2)

## Extracting Specific Layer features 
(Don't depend on sequential or functional)

In [ ]:
# here model is trained functional model with 
# last dense layer being output
model_b = keras.Model(inputs = model.inputs,
                      outputs=[model.layers[-2].output])
feature = model_b.predict(x_train)
print(feature.shape)

(60000, 512)


In [ ]:
model_a = keras.Sequential()
model_a.add(layers.Input(shape=(28*28)))
model_a.add(layers.Dense(512, activation='relu'))
model_a.add(layers.Dense(256, activation='relu', name='my_layer'))
model_a.add(layers.Dense(10))

# Not trained sequential model_a with
# last dense layer not part of layers

model_b = keras.Model(inputs=model_a.inputs,
                      outputs=[model_a.layers[-2].output])
feature = model_b.predict(x_train)
print(feature.shape)


model_b = keras.Model(inputs=model_a.inputs,
                      outputs=[model_a.get_layer('my_layer').output])
feature = model_b.predict(x_train)
print(feature.shape)

(60000, 256)
(60000, 256)


## Suggestions